In [1]:
import numpy as np
np.random.seed(1337)
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque
import random
import urllib, json
import re
from datetime import datetime
import requests


Using TensorFlow backend.


In [2]:
memory = deque(maxlen=2000)
gamma = 0.95  
epsilon = 0.1  # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.99
learning_rate = 0.001
batch_size = 32
memory = []
action_hist = []
class Account:
    cash = 100000
    amount_held = 0
    def calc_value(self,current_price):
        return self.amount_held * current_price
account = Account()

In [3]:
def build_network(input_sz):
    model = Sequential()
    model.add(Dense(125,input_dim=input_sz, activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(2, activation='linear'))
    try:
        model.load_weights("rltrader.h5")
        print "LOADEd weights"
    except:
        pass
    model.compile(loss='mse',optimizer=Adam(lr=learning_rate)) 
    return model

In [4]:
def get_data(symbol):
    q = deque(maxlen=3)
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol="+symbol+"&apikey=TOGGSCK5JLEEJTGL"
    url_rsi = "https://www.alphavantage.co/query?function=RSI&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_sma = "https://www.alphavantage.co/query?function=SMA&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_cci = "https://www.alphavantage.co/query?function=CCI&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_macd = "https://www.alphavantage.co/query?function=MACD&symbol="+symbol+"&interval=daily&series_type=open&apikey=TOGGSCK5JLEEJTGL"
    response_prices = requests.get(url)
    response_rsi = requests.get(url_rsi)
    response_sma = requests.get(url_sma)
    response_cci = requests.get(url_cci)
    response_macd = requests.get(url_macd)
    
    data = response_prices.json()
    data_rsi = response_rsi.json()
    data_sma = response_sma.json()
    data_cci = response_cci.json()
    data_macd =  response_macd.json()
    
    
    data = data["Time Series (Daily)"]
#     print(data_rsi)
    data_rsi = data_rsi["Technical Analysis: RSI"]
    data_sma = data_sma["Technical Analysis: SMA"]
    data_cci = data_cci["Technical Analysis: CCI"]
    data_macd = data_macd["Technical Analysis: MACD"]
    
    ds = {}
    ds["date"] = []
    ds["close"] = []
    ds["trend"] = []
#     ds["open"] = []
#     ds["volume"] = []
#     ds["high"] = []
#     ds["low"] = []
    ds["rsi"] = []
    ds["sma"] = []
    ds["cci"] = []
    ds["macd"] = []
    
#     i = 0
    q.append(1)
    for time_point in data:
        try:
            ds["date"] += [time_point]
            ds["close"] += [float(data[time_point]["4. close"])]
            ds["trend"] += [(float(data[time_point]["4. close"])-(sum(list(q))/len(list(q))))/(sum(list(q))/len(list(q)))]
            last_close = float(data[time_point]["4. close"])
            if last_close <= 0:
                q.append(1)
            else:
                q.append(last_close)
#             ds["open"] += [float(data[time_point]["1. open"])]
#             ds["volume"] += [float(data[time_point]["5. volume"])]
#             ds["high"] += [float(data[time_point]["2. high"])]
#             ds["low"] += [float(data[time_point][ "3. low"])]
        except KeyError as e:
            continue
        try:
            ds["macd"] += [float(data_macd[time_point]["MACD"])]
        except KeyError as e:
            ds["macd"] += [sum(ds["macd"])/len(ds["macd"])]
        try:
            ds["rsi"] += [float(data_rsi[time_point]["RSI"])]
        except KeyError as e:
            ds["rsi"] += [sum(ds["rsi"])/len(ds["rsi"])]
        try:
            ds["sma"] += [float(data_sma[time_point]["SMA"])]
        except KeyError as e:
            ds["sma"] += [sum(ds["sma"])/len(ds["sma"])]
        try:
            ds["cci"] += [float(data_cci[time_point]["CCI"])]
        except KeyError as e:
            ds["cci"] += [sum(ds["cci"])/len(ds["cci"])]
    
    df=pd.DataFrame(ds)
    df = df.sort_values(by='date')
    df = df.reset_index(drop=True)
    return df
        


In [5]:
def remember(s,a,r,sp):
    memory.append((s,a,r,sp))

In [6]:
def update(memory):
#     Take random sample from memory
    batch = random.sample(memory, batch_size) 
#     Train network on batch
    for s, a, r, sp in batch:
#       if state is final there is no future rewards
        target = r
#       discounted future rewards
    
        target = r + gamma * \
           np.amax(model.predict(sp)[0])
                
#       Get pred then replace action with our target
        target_f = model.predict(s)
        target_f[0][a] = target
        model.fit(s, target_f, epochs=1, verbose=0)

In [7]:
def get_act(obs):
    
#     if np.random.rand() <= epsilon:
# #         print "random"
# #         # The agent acts randomly
# #         epsilon = epsilon*epsilon_decay
#         act = int(random.uniform(0, 2))
#         return act
#     print "optimal"
    # Predict the reward value based on the given state
    act_values = model.predict(obs)

    # Pick the action based on the predicted reward
    return np.argmax(act_values[0])

In [8]:
int(random.uniform(0, 2))

0

In [9]:

def act(act,day):
    current_val = account.calc_value(day[1]["close"])
    i = day[0]
    future = df.close.iloc[i+1:i+3].mean()
    if act == 1:
        buy["buy"] += [day[1]["close"]]
        buy["date"] += [df_d.date.iloc[day[0]]]
        if (account.cash - (day[1]["close"]*5)) >= 0:
            account.amount_held =  account.amount_held + 5
            account.cash = account.cash - (5*day[1]["close"])
        if ((future - day[1]["close"])/day[1]["close"]) >= 0:
            return 1
        else:
            return -1
#             if (account.calc_value(future) - current_val) > 0:
#                 return 1
#             else:
#                 return -1
    #         Buy some shares reward is if we make money in a week
    else:
        
        sell["sell"] += [day[1]["close"]]
        sell["date"] += [df_d.date.iloc[day[0]]]
        to_sell =int((random.random()*account.amount_held))
        if account.amount_held > 0:
            account.cash = account.cash + (to_sell*day[1]["close"])
            account.amount_held = account.amount_held -to_sell
        if -((future - day[1]["close"])/day[1]["close"]) > 0:
#             print "Sell" , (future - day[1]["close"])/day[1]["close"]
            return 1
        else:
            return -1
#         if (account.calc_value(future) - current_val) < 0:
#             return 1
#         else:
#             return -1
    
    return -0.05
#         Sell some shares reward is if we avoid losing money in a week

In [10]:

#     print d[1]["date"]


In [11]:
range(0,1)

[0]

In [ ]:
# model = build_network(len(df.columns))
# prv_obs = None
feats = ["trend","rsi","cci","macd"]
buy = {}
buy["buy"] = []
buy["date"] = []
sell = {}
sell["sell"] = []
sell["date"] = []
worth = {}
worth["worth"]=[]
gl = []
epochs = 5
df_d = None
codes = ["TSLA","MSFT","ASX:ANZ","NVDA"]
for epoch in range(0,epochs):
    epsilon = epsilon*((epochs-epoch)/epochs)
    buy = {}
    buy["buy"] = []
    buy["date"] = []
    sell = {}
    sell["sell"] = []
    sell["date"] = []
    worth = {}
    worth["worth"]=[]
    gl = []
    i = 0
    memory = deque(maxlen=2000)
    account = Account()
    lets_trade = random.choice(codes)
#     lets_trade = "TSLA"
    print lets_trade
    df = get_data(lets_trade)
    df_d = df[["date","close"]]
    del df["date"]
    model = build_network(len(df[feats].columns))
    prv_obs = None

    for day in df.iterrows():
        i = 1 + i
    #     print i
        if i == len(df) - 11:
            model.save_weights("rltrader.h5")
            print("Episode finished after {} timesteps".format(i+1))
            break
        obs = day[1]
        obs_values = np.reshape(obs[feats].values,[1,len(df[feats].columns)])
        action = get_act(obs_values )
        reward = act(action,day)
        if action == 1:
            action_hist += ["buy"] 
        else:
            action_hist += ["sell"]
        if i % 100 == 0:
            print reward
            if action == 1:
                print "Buying" 
            else:
                print "Selling"
            print "day" , i
            print "cash",account.cash
            print "shares held",account.amount_held
            print "net worth",account.calc_value(day[1]["close"]) + account.cash
        worth["worth"] += [account.calc_value(day[1]["close"]) + account.cash]
    #     print(action)
        reward = act(action,day)
    #     print reward
        gl += [(account.calc_value(day[1]["close"]) + account.cash) - 100000]
        if prv_obs is not None and reward is not None:
            remember(prv_obs,action,reward,obs_values)
        prv_obs = obs_values  
    #     if done:
    #         model.save_weights("model.h5")
    #         print("Episode finished after {} timesteps".format(t+1))
    #         break
        if len(memory) >= batch_size:
            update(memory)
    print ((sum(gl)/len(gl))/100000)*100

NVDA
LOADEd weights
-1
Buying
day 100
cash 89116.68
shares held 396
net worth 131290.68
-1
Buying
day 200
cash 92680.03
shares held 767
net worth 141484.24
1
Buying
day 300
cash 41687.13
shares held 1767
net worth 126944.88
1
Buying
day 400
cash 277.780000001
shares held 2317
net worth 203895.74
1
Buying
day 500
cash 33921.8
shares held 596
net worth 73794.2
-1
Buying
day 600
cash 30738.8
shares held 136
net worth 35502.88
-1
Selling
day 700
cash 34932.45
shares held 1
net worth 34941.1
1
Buying
day 800
cash 34253.73
shares held 16
net worth 34454.85
1
Selling
day 900
cash 36814.647
shares held 1
net worth 36835.107
-1
Selling
day 1000
cash 36724.315
shares held 1
net worth 36746.115
-1
Buying
day 1100
cash 35635.885
shares held 36
net worth 36408.445
1
Selling
day 1200
cash 36185.815
shares held 1
net worth 36200.135
-1
Buying
day 1300
cash 31981.455
shares held 156
net worth 36326.055
1
Selling
day 1400
cash 35833.385
shares held 1
net worth 35860.655
-1
Buying
day 1500
cash 24021.29

In [ ]:
print ((sum(gl)/len(gl))/100000)*100
# df_d
print (133605.115/100000)*100
# print action_hist

In [ ]:
df_buys = pd.DataFrame(buy)
df_sells = pd.DataFrame(sell)
df_worth = pd.DataFrame(worth)

In [ ]:
df_d.plot()
df_worth.plot()


In [ ]:
df_buys.plot(style=".")

In [ ]:
# df_d.plot()
df_sells.plot(style=".")

In [ ]:
df_d.date = pd.to_datetime(df_d.date,format='%Y-%m-%d')
df_d = df_d.set_index("date")
df_buys.date = pd.to_datetime(df_buys.date,format='%Y-%m-%d')
df_buys = df_buys.set_index("date")
df_sells.date = pd.to_datetime(df_sells.date,format='%Y-%m-%d')
df_sells = df_sells.set_index("date")
ax = df_d.plot()

ax = df_d.plot()
ax = df_buys.plot(ax=ax)
df_sells.plot(ax=ax)
# # df_sells.groupby(df_sells["date"])
plt.show()

In [ ]:
fig = plt.figure(figsize = (50,50))
ax = fig.gca()
df.rsi.plot(ax=ax)

In [ ]:
fig = plt.figure(figsize = (50,50))
ax = fig.gca()
ax = df_d.plot(ax=ax)
ax = df_buys.plot(style=".",ax=ax)
df_sells.plot(style=".",ax=ax)
# # df_sells.groupby(df_sells["date"])
plt.show()

In [ ]:
df_d

In [ ]:
%%bash
which python